# Our Task: Find similar company names

Now that we have completed our Python Primer, let's try to complete a real task, while at the same time trying to practice loops, iterations, and other Python functionality that we studied.

## Your high level task

You are given a list of names. You know that the same entity in the list has different representations. You want to find duplicate companies in the data.

As a concrete example, open the file under `data/restaurant-names.txt`. This contains a list of restaurant names, extracted from the [NYC Restaurant inspection data set](https://data.cityofnewyork.us/Health/DOHMH-New-York-City-Restaurant-Inspection-Results/xx67-kt59/data) (available online). The Department of Health has been doing a decent, but not perfect, job in recording the company names. Therefore, the same restaurant appears under different names. **Your task is to find "almost duplicate" entries in order to decide whether they correspond to the same business.**


In [1]:
!head -10 ../data/restaurant-names.txt

#1 GARDEN CHINESE
#1 ME. NICK'S
#1 SABOR LATINO RESTAURANT
$1.25 PIZZA
''U'' LIKE CHINESE RESTAURANT
''W'' CAFE
'WICHCRAFT
(LEWIS DRUG STORE) LOCANDA VINI E OLII
(LIBRARY)  FOUR & TWENTY BLACKBIRDS
(PUBLIC FARE) 81ST STREET AND CENTRAL PARK WEST (DELACORTE THEATRE)


In [2]:
!tail -5 ../data/restaurant-names.txt

ZUTTO
ZUZU RAMEN
ZYMI BAR & GRILL
ZZ CLAM BAR
ZZ'S PIZZA & GRILL


## Matching Company Names

Quite often, in our data, we have entries represented as strings that refer to the same entity but have different string representations (e.g., McDonald's vs McDonalds vs McDonald‎). We want to write code that will help in the task of finding such similar entries in our data.

Our task can be broken in the following tasks:
* **Step 1**: Read the data from a file into a list of strings in memory. We have a data set under the `data` folder: The list of unique restaurant names from the NYC Restaurant Inspection dataset (`data/restaurant-names.txt`). We need to write Python code that will read th file and return a list of strings that are the company names.
* **Step 2**: We will then need to figure out how to compare two strings and find their similarity. For that, we will write a function that takes as input **two** strings, and returns back their similarities. We will explore multiple ways of writing that function, using various library functions.
* **Step 3**: We will need to write a function that takes as input a company name, and returns back a list of matching company names, together with their similarity. Ideally, we would like to also sort the list based on the similarity (highest similarity metrics on top). As part of our learning process, we will use the _list comprehension_ approach to create the list. We will also use tuples as the elements of the list, so that the list contain elements such as `[("McDonalds", 0.88), ("McDonald's", 0.81),....]`.
* **Step 4**: In the final step, we will just perform the similarity computation across all companies in the dataset.

### STEP 1: Read the list of names from a file and create a list of names

In [3]:
# STEP 1: Read the list of names from a file and create a list of names
filename = '../data/restaurant-names.txt'

# We open the filename for reading
f = open(filename, 'r')
# Read the file into memory
content = f.read()
# Content is a big string, with one restaurant per line
# so we split it into lines and create a list with the restaurant names
restaurants = content.split('\n')

In [4]:
len(restaurants)

20444

### STEP 2: Implement the similarity function

### Computing the similarity between two strings

There are many ways that we can calculate the similarity between two strings. For our case, we will focus on a few similarity metrics that already have implementations in Python. 

##### Some commonly used similarity metrics

* [Sequence matching](https://docs.python.org/3.6/library/difflib.html) (part of standard Python) ([example](http://stackoverflow.com/questions/17388213/find-the-similarity-percent-between-two-strings))
* [Edit distance](https://en.wikipedia.org/wiki/Edit_distance) ([Python Jellyfish Library](https://github.com/jamesturk/jellyfish))
* [N-gram distance](http://pythonhosted.org/ngram/tutorial.html#comparing-and-searching-strings)


#### STEP 2.a: Let's figure out how we can install the libraries...

In [25]:
# Edit distance
!sudo -H pip3 install -U jellyfish

Requirement already up-to-date: jellyfish in /usr/local/lib/python3.6/dist-packages (0.7.1)


In [26]:
# Ngram
!sudo -H pip3 install -U ngram

Requirement already up-to-date: ngram in /usr/local/lib/python3.6/dist-packages (3.3.2)


#### STEP 2.b: Now let's test our similarity functions with various examples

Once we have installed the necessary libraries for our project, we proceed to `import` them and test the functions.

In [27]:
import jellyfish

##### Edit Distance

From Wikipedia:

The [edit distance](https://en.wikipedia.org/wiki/Edit_distance) _ is a way of quantifying how dissimilar two strings (e.g., words) are to one another by counting the minimum number of operations required to transform one string into the other._. 

The Levenshtein distance between "kitten" and "sitting" is 3. A minimal edit script that transforms the former into the latter is:

* kitten → sitten (substitution of "s" for "k")
* sitten → sittin (substitution of "i" for "e")
* sittin → sitting (insertion of "g" at the end).

In [28]:
jellyfish.levenshtein_distance('kitten', 'sitting')

3

Let's try a few more examples

In [29]:
jellyfish.levenshtein_distance('Ipeirotis', 'Iperiotos')

3

In [30]:
jellyfish.levenshtein_distance('Starbucks', 'Starbacks')

1

In [12]:
jellyfish.levenshtein_distance('Starbucks', 'Starbuck')

1

In [13]:
jellyfish.levenshtein_distance('Starbucks', 'Wendys')

8

##### Demerau Levenshtein distance

The Demerau Levenshtein distance also allows for  transposition of two adjacent characters.

In [14]:
jellyfish.levenshtein_distance('Ipeirotis', 'Iperiotis')

2

In [15]:
jellyfish.damerau_levenshtein_distance('Ipeirotis', 'Iperiotis')

1

In [16]:
jellyfish.damerau_levenshtein_distance('Starbucks', 'Starbucsk')

1

In [17]:
jellyfish.levenshtein_distance('Starbucks', 'Starbucsk')

2

###### Jaro–Winkler distance

[Jaro–Winkler distance](https://en.wikipedia.org/wiki/Jaro%E2%80%93Winkler_distance) is a string metric for measuring the edit distance between two sequences. Informally, the **Jaro** distance between two words is the minimum number of single-character transpositions required to change one word into the other; the **Jaro–Winkler** distance  gives more favourable ratings to strings that match from the beginning.


In [18]:
jellyfish.jaro_winkler('Starbucks', 'Starbarbr')

0.8222222222222222

In [19]:
jellyfish.jaro_winkler('Starbucks', 'Milwbucks')

0.7037037037037037

In [35]:
jellyfish.jaro_winkler('Borja', 'Deniz')

0.0

##### Soundex

[Soundex](https://en.wikipedia.org/wiki/Soundex) is a phonetic algorithm for indexing names by sound, as pronounced in English. The goal is for homophones to be encoded to the same representation so that they can be matched despite minor differences in spelling. 

Using this algorithm, both "Robert" and "Rupert" return the same string "R163" while "Rubin" yields "R150". "Ashcraft" and "Ashcroft" both yield "A261". "Tymczak" yields "T522" not "T520" (the chars 'z' and 'k' in the name are coded as 2 twice since a vowel lies in between them). "Pfister" yields "P236" not "P123" (the first two letters have the same number and are coded once as 'P').

In [20]:
jellyfish.soundex('Robert')

'R163'

In [21]:
jellyfish.soundex('Rupert')

'R163'

In [22]:
jellyfish.soundex('Ashcroft')

'A226'

In [23]:
jellyfish.soundex('Ashcraft')

'A226'

In [24]:
jellyfish.soundex('Papadopoylis')

'P131'

In [25]:
jellyfish.soundex('Papadopoulos')

'P131'

In [33]:
jellyfish.soundex('Borja')

'B620'

#### Ngrams

With the n-gram similarity score, we split the word into sequences of n consecutive characters (n-grams) and then compare the sets of n-grams between the two words. For example, the name "Panos" has the following 2-grams: "Pa", "an", "no", "os". (We can also add "#P" and "s#" if we want to capture the prefix and suffixes.) Strings that share a large number of q-grams are often similar.

In [36]:
import ngram

In [37]:
ngram.NGram.compare('Ipeirotis','Iperotis',N=2)

0.7272727272727273

In [28]:
ngram.NGram.compare('Ipeirotis','Iperotis',N=1)

0.8888888888888888

In [29]:
ngram.NGram.compare('New York University','New York Universty',N=2)

0.8571428571428571

In [30]:
ngram.NGram.compare('New York University','University of New York',N=1)

0.8636363636363636

In [31]:
ngram.NGram.compare('New York University','Columbia Universty',N=2)

0.3

#### Task 1: Create a function that takes as input two strings and returns their similarity

Given the experience with the metrics above, we want now to create a function that takes as input a string and returns their similarity. Our key requirement is for the similarity metric to be between 0 and 1, with 0 meaning no similarity, and 1 corresponding to identical strings. Some of the similarity functions above would fit right in, others will need some work. 

In [47]:
# For n-gram similarity it is very simple, we just return the result
import ngram
def computeSimilarity_ngram(str1, str2):
    similarity = ngram.NGram.compare(str1,str2,N=3)
    return similarity

computeSimilarity_ngram("New York University", "New York Univ")
computeSimilarity_ngram("New York University", "Columbia Univ")

0.09090909090909091

In [48]:
# For edit distance
import jellyfish
def computeSimilarity_editdistance(str1, str2):
    # Compute the maximum length of the two strings, to normalize our distance
    maxlength = max( len(str1), len(str2))
    # Compute the edit distance between two strings
    distance = jellyfish.levenshtein_distance(str1, str2)
    similarity = 1 - distance/maxlength
    return similarity

computeSimilarity_editdistance("New York University", "New York Univ")
computeSimilarity_editdistance("New York University", "Columbia Univ")

0.26315789473684215

In [49]:
# For soundex
import jellyfish
def computeSimilarity_soundex(str1, str2):
    soundex1 = jellyfish.soundex(str1)
    soundex2 = jellyfish.soundex(str2)
    if soundex1 == soundex2:
        return 1.0
    else: 
        return 0.0
    
computeSimilarity_soundex("New York University", "New York Univ")

1.0

#### Task 2: Modify the functions above to allow for various similarity calculation methods.

We will now up our game, and return back the results of the comparison using various methods. The `method` parameter defines the metric that we will use

In [50]:
def computeSimilarity(str1, str2, method):
    if method == 'ngram3':
        return computeSimilarity_ngram(str1, str2)
    elif method == 'edit_distance':
        return computeSimilarity_editdistance(str1, str2)
    elif method == 'soundex':
        return computeSimilarity_soundex(str1, str2)
    else:
        return "Method not supported"

In [51]:
computeSimilarity("New York University", "New York Univ", 'ngram3')

0.5652173913043478

In [52]:
computeSimilarity("New York University", "New York Univ", 'edit_distance')

0.6842105263157895

In [53]:
# Most of the time we are going to compute all similarity metrics
# and return back a dictionary with all the metrics
def computeSimilarity(str1, str2):
    results = {
        'ngram': computeSimilarity_ngram(str1, str2),
        'edit_distance' : computeSimilarity_editdistance(str1, str2),
        'soundex': computeSimilarity_soundex(str1, str2) 
    }
    results['average'] = sum(results.values()) / len(results)
    return results

In [54]:
computeSimilarity("New York University", "New York Univ")

{'average': 0.749809305873379,
 'edit_distance': 0.6842105263157895,
 'ngram': 0.5652173913043478,
 'soundex': 1.0}

### STEP 3: Compute similarity of a company against a list of company names

We now create a function that accepts a company name and a list of companies, and computes their similarity. This part will get us to exercise our for-loops, and also illustrate how we can use lists and tuples.

**Sorting a list of tuples**:_This part is a little bit advanced for now, so I will just give the code below. (Solution taken from http://stackoverflow.com/questions/3121979/how-to-sort-list-tuple-of-lists-tuples). Here is a small example below, which we will reuse in our function:_

In [55]:
data = [("Panos",0.5), ("Peter",0.6), ("Pan", 0.4)]
data.sort(key=lambda tupl: tupl[1], reverse=True)  # sorts in place, in descending order, based on the second element of the tuple
print(data)

[('Peter', 0.6), ('Panos', 0.5), ('Pan', 0.4)]


In [41]:
# STEP 3: We now create a function that accepts a company name
# and a list of companies, and computes their similarity
# We have a 'top' parameter (by default set to be 5)
# that restricts the results to only the most similar 
# string pairs. We also define a parameter "method" that defines 
# what is the similarity method that we want to use. We also define a 
# similarity threshold for keeping only results with sufficient similarity

def companySimilarity(query, companyList, top = 5, method = 'average', sim_threshold = 0.25):
    # We will use a list to store the similar matches
    results = []

    # Go through all the restaurants
    for c in companyList:
        # We compute the similarities (all metrics)
        # between the string "query" and the string "c"
        # which is the variable that iterates over the list "companyList"
        similarities = computeSimilarity(query, c)
        # If the ngram similarity is above 0.25
        if similarities[method]>sim_threshold:
            # Add in results the matching restaurant name r
            # and the similarity
            results.append( (c, similarities[method]) )
        # ALTERNATIVELY, we can average all similarities
        # and r

    # This list contains the matches. The list contains a list
    # of tuples (company name, similarity)
    # We sort in decreasing order of similarity
    results.sort(key=lambda tupl: tupl[1], reverse=True)
    
    # We return only the top results
    return results[:top]    

In [42]:
query = 'MACDONALDS'
companySimilarity(query, restaurants, 
                  top = 5, method = 'ngram', sim_threshold = 0.25)


[('MCDONALDS', 0.6428571428571429),
 ('MC DONALDS', 0.5),
 ("MCDONALD'S", 0.4117647058823529),
 ('MCDONALD', 0.375),
 ('MCDONALDS 17754', 0.3181818181818182)]

In [43]:
query = 'MACDONALDS'
companySimilarity(query, restaurants, 
                  top = 5, method = 'average', sim_threshold = 0.25)


[('MCDONALDS', 0.8476190476190476),
 ('MC DONALDS', 0.7666666666666666),
 ("MCDONALD'S", 0.7372549019607844),
 ('MCDONALD', 0.725),
 ("MC DONALD'S", 0.6810207336523125)]

In [44]:
query = 'STARBUCKS'
companySimilarity(query, restaurants, 
                  top = 5, method = 'ngram', sim_threshold = 0.25)


[('STARBUCKS', 1.0),
 ('STARBUCKS 22420', 0.47368421052631576),
 ('STARBUCKS COFFEE', 0.45),
 ('STARBUCKS # 14840', 0.42857142857142855),
 ('STARBUCKS COFFEE#7462', 0.36)]

In [45]:
query = 'STARBUCKS'
companySimilarity(query, restaurants, 
                  top = 20, method = 'average', sim_threshold = 0.25)


[('STARBUCKS', 1.0),
 ('STARBUCKS 22420', 0.6912280701754385),
 ('STARBUCKS COFFEE', 0.6708333333333334),
 ('STARBUCKS # 14840', 0.6526610644257703),
 ('STARBUCKS COFFEE#7462', 0.5961904761904763),
 ('STARBUCKS COFFEE #3438', 0.585081585081585),
 ('STARBUCKS COFFEE #7344', 0.585081585081585),
 ('STARBUCKS COFFEE #7358', 0.585081585081585),
 ('STARBUCKS COFFEE #7416', 0.585081585081585),
 ('STARBUCKS COFFEE #7682', 0.585081585081585),
 ('STARBUCKS COFFEE #7826', 0.585081585081585),
 ('STARBUCKS COFFEE #9282', 0.585081585081585),
 ('STARBUCKS COFFEE #9722', 0.585081585081585),
 ('STARBUCKS COFFEE# 7426', 0.585081585081585),
 ('STARBUCKS (STORE 16628)', 0.5748792270531401),
 ('STARBUCKS COFFEE # 7463', 0.5748792270531401),
 ('STARBUCKS COFFEE # 7540', 0.5748792270531401),
 ('STARBUCKS COFFEE #14240', 0.5748792270531401),
 ('STARBUCKS COFFEE #18509', 0.5748792270531401),
 ('STARBUCKS COFFEE #20679', 0.5748792270531401)]

### Step 4: Perform the similarity computation across all companies in the dataset.

In [46]:
# STEP 4: We are almost done. We now just go through all the companies in the list
# and we call the companySimilarity function that computes the similar company names
# for all the companies in the list. We store the results in a dictionary, with the 
# key being the company name, and the value being a "list of tuples" with the 
# similar company names and the corresponding similarity value.


In [47]:
matches = 0 
for q in restaurants:
    results = companySimilarity(q, restaurants, 
                  top = 5, method = 'average', sim_threshold = 0.5)
    # We will print only non-identical matches (remember the top
    # match is  the restaurant matching with itself)
    if (len(results)>1):
        print(q, "===>", results[1:])
        print("\n")
        matches = matches + 1
    if matches > 10:
        break # stop after finding 10 restaurants with matches

#1 SABOR LATINO RESTAURANT ===> [('SABOR LATINO RESTAURANT', 0.5504273504273504)]


''U'' LIKE CHINESE RESTAURANT ===> [('U LIKE CHINESE RESTAURANT', 0.5226504394861392)]


'WICHCRAFT ===> [('WICHCRAFT', 0.5142857142857143)]


1001 NIGHTS ===> [('1001 NIGHTS CAFE', 0.7458333333333332)]


1001 NIGHTS CAFE ===> [('1001 NIGHTS', 0.7458333333333332)]


1005 CATERING ===> [('1061 CATERING LLC', 0.6690196078431372)]


101 CAFE ===> [('19 CAFE', 0.7371794871794872)]


101 RESTAURANT AND BAR ===> [('111 RESTAURANT', 0.6567398119122257), ('18 RESTAURANT', 0.6300940438871473), ('508 RESTAURANT AND BAR', 0.5214211076280041)]


102 NOODLES TOWN RESTAURANT ===> [('16 HANDLES FROZEN YOGURT', 0.5024691358024692)]


1020 BAR ===> [('169 BAR', 0.6607142857142857)]


1028 BAR & RESTAURANT EL SALVADORENO  ===> [('BAR & RESTAURANT EL SALVADORENO', 0.5040854808296669)]




In [48]:
matches = 0

for q in restaurants:
    results = companySimilarity(q, restaurants, 
                  top = 5, method = 'average', sim_threshold = 0.5)
    # We will print only non-identical matches (remember the top
    # match is  the restaurant matching with itself)
    if (len(results)>1):
        for r in results[1:]:
            print(f"{q}\t{r[0]}\t{r[1]}")
            matches = matches + 1
    if matches>10: 
        break # We stop after 10 matches 

#1 SABOR LATINO RESTAURANT	SABOR LATINO RESTAURANT	0.5504273504273504
''U'' LIKE CHINESE RESTAURANT	U LIKE CHINESE RESTAURANT	0.5226504394861392
'WICHCRAFT	WICHCRAFT	0.5142857142857143
1001 NIGHTS	1001 NIGHTS CAFE	0.7458333333333332
1001 NIGHTS CAFE	1001 NIGHTS	0.7458333333333332
1005 CATERING	1061 CATERING LLC	0.6690196078431372
101 CAFE	19 CAFE	0.7371794871794872
101 RESTAURANT AND BAR	111 RESTAURANT	0.6567398119122257
101 RESTAURANT AND BAR	18 RESTAURANT	0.6300940438871473
101 RESTAURANT AND BAR	508 RESTAURANT AND BAR	0.5214211076280041
102 NOODLES TOWN RESTAURANT	16 HANDLES FROZEN YOGURT	0.5024691358024692
